In [1]:
import pandas as pd
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import time
from datetime import datetime
import re

In [2]:
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
opciones.add_argument('--incognito')


In [3]:
from selenium import webdriver
PATH = "/driver/chromedriver"
from selenium.webdriver.common.by import By
driver = webdriver.Chrome(opciones)

In [4]:
url = 'https://violenciagenero.igualdad.gob.es/sensibilizacionConcienciacion/campannas/violenciaGobierno/home.htm'

driver.get(url)

In [5]:
urls = driver.find_elements(By.XPATH, '//*[@id="contenido"]/div[3]/div/ul/li/a')

urlss = [i.get_attribute('href') for i in urls]

urlss = urlss[:9]

urlss

['https://violenciagenero.igualdad.gob.es/sensibilizacionConcienciacion/campannas/violenciaGobierno/ESTARVIVAESPA/vgeneroImg.htm',
 'https://violenciagenero.igualdad.gob.es/sensibilizacionConcienciacion/campannas/violenciaGobierno/entoncesqui/vgenero.htm',
 'https://violenciagenero.igualdad.gob.es/sensibilizacionConcienciacion/campannas/violenciaGobierno/vivas_libres/vgenero.htm',
 'https://violenciagenero.igualdad.gob.es/sensibilizacionConcienciacion/campannas/violenciaGobierno/Juntas/vgenero.htm',
 'https://violenciagenero.igualdad.gob.es/sensibilizacionConcienciacion/campannas/violenciaGobierno/BuenosTratos/vgenero.htm',
 'https://violenciagenero.igualdad.gob.es/sensibilizacionConcienciacion/campannas/violenciaGobierno/CeroDieciseis/vgenero.htm',
 'https://violenciagenero.igualdad.gob.es/sensibilizacionConcienciacion/campannas/violenciaGobierno/ElSexoEsunSi/home.htm',
 'https://violenciagenero.igualdad.gob.es/sensibilizacionConcienciacion/campannas/violenciaGobierno/MachismoEsViolen

In [6]:
def extraer_fecha(url):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(3)
    driver.find_element(By.XPATH, '//*[@id="mensaje"]/div[1]/a').click() # Clickamos sobre el mensaje de inicio para entrar en la web
    time.sleep(3)
    frame = driver.find_element(By.CSS_SELECTOR, 'iframe.player') # El siguiente paso es darle al play en la miniatura
    driver.switch_to.frame(frame)    # Para ello hay que entrar con el driver en el propio frame del vídeo
    time.sleep(3)
    driver.find_element(By.XPATH, '//*[@id="movie_player"]/div[4]/button').click() # Ahora sí le damos al play
    driver.find_element(By.CSS_SELECTOR, '#movie_player > div.ytp-chrome-bottom > div.ytp-chrome-controls > div.ytp-right-controls > a').click() # Nos redirige a Youtube
    driver.switch_to.window(driver.window_handles[1])  # Para cambiar de ventana, ya que hemos abierto una nueva
    time.sleep(3)
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[6]/div[1]/ytd-button-renderer[2]/yt-button-shape').click()  # Aceptamos cookies
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, '#movie_player > div.ytp-chrome-bottom > div.ytp-chrome-controls > div.ytp-left-controls > button').click()   # Pausamos el vídeo
    time.sleep(3)
    driver.find_element(By.XPATH, '//*[@id="expand"]').click()    # Desplegamos para ver la fecha
    fecha = [driver.find_element(By.XPATH, '//*[@id="info"]/span[3]').text]   # Extraemos la fecha
    driver.quit()
    return fecha

In [7]:
campañas = {}

for i in range(len(urlss)):
    campañas[f"campaña{i + 1}"] = urlss[i]
campañas

{'campaña1': 'https://violenciagenero.igualdad.gob.es/sensibilizacionConcienciacion/campannas/violenciaGobierno/ESTARVIVAESPA/vgeneroImg.htm',
 'campaña2': 'https://violenciagenero.igualdad.gob.es/sensibilizacionConcienciacion/campannas/violenciaGobierno/entoncesqui/vgenero.htm',
 'campaña3': 'https://violenciagenero.igualdad.gob.es/sensibilizacionConcienciacion/campannas/violenciaGobierno/vivas_libres/vgenero.htm',
 'campaña4': 'https://violenciagenero.igualdad.gob.es/sensibilizacionConcienciacion/campannas/violenciaGobierno/Juntas/vgenero.htm',
 'campaña5': 'https://violenciagenero.igualdad.gob.es/sensibilizacionConcienciacion/campannas/violenciaGobierno/BuenosTratos/vgenero.htm',
 'campaña6': 'https://violenciagenero.igualdad.gob.es/sensibilizacionConcienciacion/campannas/violenciaGobierno/CeroDieciseis/vgenero.htm',
 'campaña7': 'https://violenciagenero.igualdad.gob.es/sensibilizacionConcienciacion/campannas/violenciaGobierno/ElSexoEsunSi/home.htm',
 'campaña8': 'https://violenciag

In [8]:
fechas = []
for campaña, url in campañas.items():
    fechas.append(extraer_fecha(url))

In [9]:
fechas

[['13 Jul 2023'],
 ['21 Nov 2022'],
 ['8 Aug 2022'],
 ['26 Nov 2021'],
 ['25 Nov 2021'],
 ['13 Oct 2021'],
 ['25 May 2021'],
 ['14 Apr 2021'],
 ['25 Nov 2020']]

In [10]:
fecha_campaña = [datetime.strptime(fecha[0], '%d %b %Y').date() for fecha in fechas]

fecha_campaña

    

[datetime.date(2023, 7, 13),
 datetime.date(2022, 11, 21),
 datetime.date(2022, 8, 8),
 datetime.date(2021, 11, 26),
 datetime.date(2021, 11, 25),
 datetime.date(2021, 10, 13),
 datetime.date(2021, 5, 25),
 datetime.date(2021, 4, 14),
 datetime.date(2020, 11, 25)]

In [11]:
campañas_vg = pd.DataFrame({'Año': [i.year for i in fecha_campaña], 
                            'Mes': [i.month for i in fecha_campaña], 
                            'URL': campañas.values()})
campañas_vg

,Año,Mes,URL
0,2023,7,https://violenciagenero.igualdad.gob.es/sensib...
1,2022,11,https://violenciagenero.igualdad.gob.es/sensib...
2,2022,8,https://violenciagenero.igualdad.gob.es/sensib...
3,2021,11,https://violenciagenero.igualdad.gob.es/sensib...
4,2021,11,https://violenciagenero.igualdad.gob.es/sensib...
5,2021,10,https://violenciagenero.igualdad.gob.es/sensib...
6,2021,5,https://violenciagenero.igualdad.gob.es/sensib...
7,2021,4,https://violenciagenero.igualdad.gob.es/sensib...
8,2020,11,https://violenciagenero.igualdad.gob.es/sensib...


In [12]:
campañas_vg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Año     9 non-null      int64 
 1   Mes     9 non-null      int64 
 2   URL     9 non-null      object
dtypes: int64(2), object(1)
memory usage: 344.0+ bytes


In [18]:
# DOCUMENTAL ROCIO

url_ori = 'https://maldita.es/malditodato/20210409/llamadas-016-crecieron-marzo-documental-rocio-carrasco/'

driver = webdriver.Chrome()
driver.get(url_ori)
time.sleep(3)
driver.find_element(By.XPATH, '//*[@id="iuxh"]/div[1]/div/div[2]/button[1]').click()   # Aceptamos cookies
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="article-text"]/p[1]/a[2]').click()  #Pulsamos sobre el hipervínculo 'emisión del primer episodio el 21 de marzo'
time.sleep(3)
driver.switch_to.window(driver.window_handles[1])  # Cambiamos de ventana, ya que hemos abierto una nueva
driver.find_element(By.XPATH, '//*[@id="didomi-notice-agree-button"]').click()  # Aceptamos cookies de nuevo
time.sleep(2)
titular = driver.find_element(By.XPATH, '//*[@id="article_multisite"]/div/article/header/h1').text  # Clickamos sobre el texto donde se refleja la fecha
fecha_docu = titular.split()[-1]   # Dividimos la string por espacios y nos quedamos con el último elemento, que es la fecha
driver.quit()

In [20]:
#mes_docu = int(fecha_docu[1])
fecha_docu

'(21/03/21)'

In [25]:
fecha_docu = fecha_docu.split('/')
fecha_docu

['(21', '03', '21)']

In [27]:
mes_docu = int(fecha_docu[1])
mes_docu

3

In [28]:
año_docu = re.search(r'\d+', fecha_docu[2]).group()   # Nos quedamos solo con la parte numérica
año_docu

'21'

In [35]:
año_doc = int(datetime.strptime(año_docu, "%y").strftime("%Y"))
año_doc

2021

In [34]:
nueva_fila = pd.DataFrame({'Año': [año_doc], 'Mes': [mes_docu], 'URL': ['https://www.mitele.es/programas-tv/rocio-contar-la-verdad-para-seguir-viva/']})

campañas_vg = pd.concat([campañas_vg, nueva_fila], ignore_index=True)
campañas_vg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Año     10 non-null     int64 
 1   Mes     10 non-null     int64 
 2   URL     10 non-null     object
dtypes: int64(2), object(1)
memory usage: 368.0+ bytes


In [36]:
campañas_vg = campañas_vg.sort_values(by='Año')
campañas_vg

,Año,Mes,URL
8,2020,11,https://violenciagenero.igualdad.gob.es/sensib...
3,2021,11,https://violenciagenero.igualdad.gob.es/sensib...
4,2021,11,https://violenciagenero.igualdad.gob.es/sensib...
5,2021,10,https://violenciagenero.igualdad.gob.es/sensib...
6,2021,5,https://violenciagenero.igualdad.gob.es/sensib...
7,2021,4,https://violenciagenero.igualdad.gob.es/sensib...
9,2021,3,https://www.mitele.es/programas-tv/rocio-conta...
1,2022,11,https://violenciagenero.igualdad.gob.es/sensib...
2,2022,8,https://violenciagenero.igualdad.gob.es/sensib...
0,2023,7,https://violenciagenero.igualdad.gob.es/sensib...


In [37]:
# EXPORTAR CSV
campañas_vg.to_csv('../data/data_clean/campañas_vg.csv', index=False)
